<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/Insurance_MYV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    matthews_corrcoef,  # Added for Matthews Correlation
    roc_auc_score,      # Added for AUC score
    accuracy_score      # Added for test accuracy
)


- Imports necessary libraries for data manipulation (pandas, numpy)
- Machine learning tools from scikit-learn
- SMOTE for handling imbalanced data

In [2]:
# Step 1: Load the dataset
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify file path
file_path = '/content/drive/My Drive/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
data = pd.read_csv(file_path)

# Step 2: Explore the data
print(data.head())
print(data.info())

# Check for missing values
print(data.isnull().sum())

# Display basic statistics
print(data.describe())

Mounted at /content/drive
   Duration  Insured.age Insured.sex  Car.age  Marital  Car.use  Credit.score  \
0       366           45        Male       -1  Married  Commute         609.0   
1       182           44      Female        3  Married  Commute         575.0   
2       184           48      Female        6  Married  Commute         847.0   
3       183           71        Male        6  Married  Private         842.0   
4       183           84        Male       10  Married  Private         856.0   

  Region  Annual.miles.drive  Years.noclaims  ...  Left.turn.intensity10  \
0  Urban             6213.71              25  ...                    1.0   
1  Urban            12427.42              20  ...                   58.0   
2  Urban            12427.42              14  ...                    0.0   
3  Urban             6213.71              43  ...                    0.0   
4  Urban             6213.71              65  ...                    2.0   

   Left.turn.intensity11  Left

- Data Loading:

    - Mounts Google Drive
    - Reads the telematics dataset from CSV


- Data Exploration:

    - Shows first few rows with head()
    - Displays dataset info and statistics
    - Checks for missing values

In [3]:
# Separate features and target
X = data.drop(['NB_Claim', 'AMT_Claim'], axis=1)
y = (data['NB_Claim'] > 0).astype(int)  # Convert to binary (0 for no claim, 1 for claim)

In [4]:
# Check class balance
class_counts = y.value_counts()
print("Class distribution:")
print(class_counts)
print(f"Class balance ratio: {class_counts[0] / class_counts[1]:.2f}")

Class distribution:
NB_Claim
0    95728
1     4272
Name: count, dtype: int64
Class balance ratio: 22.41


- Separates features (X) and target (y)
- Converts claims to binary classification (0 for no claim, 1 for claim)
- Checks class distribution and imbalance

In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Define numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = ['Insured.sex', 'Marital', 'Car.use', 'Region']

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

- Splits data into training and test sets
- Creates preprocessor for both numeric and categorical features
- Standardizes numeric features
- One-hot encodes categorical features

In [7]:
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

In [8]:
# Create Random Forest classifier with balanced class weights
rf_classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1)


In [9]:
# Create the full pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', rf_classifier)
])

In [10]:
# Fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Duration', 'Insured.age', 'Car.age', 'Credit.score',
       'Annual.miles.drive', 'Years.noclaims', 'Territory',
       'Annual.pct.driven', 'Total.miles.driven', 'Pct.drive.mon',
       'Pct.drive.tue', 'Pct.drive.wed', 'Pct.drive.thr', 'Pct.drive.fri',
       'Pct.drive.sat', 'Pct.drive.sun', 'P...
       'Left.turn.intensity12', 'Right.turn.intensity08',
       'Right.turn.intensity09', 'Right.turn.intensity10',
       'Right.turn.intensity11', 'Right.turn.intensity12'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Insured.sex', 'Marital',
                                                   'Car.use', 'Region'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', n_jobs=-1,
                                        random_state=42))])

In [11]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     19130
           1       0.97      0.26      0.41       870

    accuracy                           0.97     20000
   macro avg       0.97      0.63      0.70     20000
weighted avg       0.97      0.97      0.96     20000



In [12]:
# Print the confusion matrix
print(confusion_matrix(y_test, y_pred))


[[19122     8]
 [  641   229]]


In [13]:

# Perform cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=5)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")


Cross-validation scores: [0.968   0.9677  0.96725 0.96695 0.9692 ]
Mean CV score: 0.968 (+/- 0.002)


In [ ]:
# Feature importance
feature_importance = pipeline.named_steps['classifier'].feature_importances_
feature_names = (preprocessor.named_transformers_['num'].get_feature_names_out().tolist() +
                 preprocessor.named_transformers_['cat'].get_feature_names_out().tolist())

In [ ]:
# Sort feature importances in descending order
indices = np.argsort(feature_importance)[::-1]

# Print the top 20 feature ranking
print("\nTop 20 Feature ranking:")
for f in range(min(20, len(feature_names))):
    print("%d. %s (%f)" % (f + 1, feature_names[indices[f]], feature_importance[indices[f]]))


Top 20 Feature ranking:
1. Total.miles.driven (0.103526)
2. Annual.pct.driven (0.092099)
3. Duration (0.033678)
4. Credit.score (0.032826)
5. Pct.drive.2hrs (0.030017)
6. Avgdays.week (0.030016)
7. Pct.drive.thr (0.028129)
8. Pct.drive.rush am (0.027172)
9. Pct.drive.rush pm (0.024612)
10. Pct.drive.wed (0.024558)
11. Pct.drive.mon (0.024377)
12. Pct.drive.tue (0.023702)
13. Pct.drive.fri (0.023537)
14. Brake.06miles (0.022965)
15. Pct.drive.sat (0.022950)
16. Pct.drive.sun (0.022392)
17. Car.age (0.021313)
18. Accel.06miles (0.020572)
19. Years.noclaims (0.020419)
20. Left.turn.intensity09 (0.020307)


In [ ]:
!pip install pytorch_tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [ ]:
# Select top 20 features based on the provided ranking
top_features = [
    'Total.miles.driven', 'Annual.pct.driven', 'Duration', 'Credit.score',
    'Pct.drive.2hrs', 'Avgdays.week', 'Pct.drive.thr', 'Pct.drive.rush am',
    'Pct.drive.rush pm', 'Pct.drive.wed', 'Pct.drive.mon', 'Pct.drive.tue',
    'Pct.drive.fri', 'Brake.06miles', 'Pct.drive.sat', 'Pct.drive.sun',
    'Car.age', 'Accel.06miles', 'Years.noclaims', 'Left.turn.intensity09'
]

In [ ]:
# Prepare the data
X = data[top_features]
y = (data['NB_Claim'] > 0).astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test.values)

In [ ]:
# Initialize and train TabNet
tabnet_model = TabNetClassifier(
    n_d=8, n_a=8, n_steps=3,
    gamma=1.3, n_independent=2, n_shared=2,
    cat_idxs=[], cat_dims=[],
    cat_emb_dim=1, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax',
    lambda_sparse=1e-3, momentum=0.3, clip_value=2.,
    device_name='auto'
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [ ]:
tabnet_model.fit(
    X_train_tensor.numpy(), y_train_tensor.numpy(),
    eval_set=[(X_test_tensor.numpy(), y_test_tensor.numpy())],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

epoch 0  | loss: 0.18404 | val_0_auc: 0.77508 |  0:00:05s
epoch 1  | loss: 0.15517 | val_0_auc: 0.78477 |  0:00:09s
epoch 2  | loss: 0.15385 | val_0_auc: 0.78575 |  0:00:15s
epoch 3  | loss: 0.15333 | val_0_auc: 0.78692 |  0:00:20s
epoch 4  | loss: 0.15285 | val_0_auc: 0.7868  |  0:00:24s
epoch 5  | loss: 0.15216 | val_0_auc: 0.79419 |  0:00:30s
epoch 6  | loss: 0.15212 | val_0_auc: 0.79033 |  0:00:35s
epoch 7  | loss: 0.15163 | val_0_auc: 0.7905  |  0:00:39s
epoch 8  | loss: 0.15166 | val_0_auc: 0.79377 |  0:00:44s
epoch 9  | loss: 0.15105 | val_0_auc: 0.79186 |  0:00:49s
epoch 10 | loss: 0.15111 | val_0_auc: 0.78904 |  0:00:54s
epoch 11 | loss: 0.15066 | val_0_auc: 0.79639 |  0:00:59s
epoch 12 | loss: 0.1495  | val_0_auc: 0.79603 |  0:01:04s
epoch 13 | loss: 0.14988 | val_0_auc: 0.79625 |  0:01:08s
epoch 14 | loss: 0.14894 | val_0_auc: 0.80203 |  0:01:14s
epoch 15 | loss: 0.14905 | val_0_auc: 0.79634 |  0:01:19s
epoch 16 | loss: 0.15076 | val_0_auc: 0.79501 |  0:01:23s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# Automatically detect device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#The TabNetClassifier does not have a to() method, instead it sets the device during initialization
#Remove or comment out the following line:
#tabnet_model.to(device)

# Instead, ensure you specified the correct device when you initialized the model
#If you are still encountering issues and your device is set to 'auto', you can set it using the device param
#tabnet_model = TabNetClassifier(device_name=device) # Replace device with 'cuda' or 'cpu' as needed

# Convert data to PyTorch tensors and move to the same device as the model
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Extract learned features using the model's forward function to get embeddings
with torch.no_grad():
    # Make sure network method takes device into account
    # Assuming the first element of the tuple is the desired output
    output = tabnet_model.network(X_train_tensor.to(tabnet_model.device))
    train_features = output[0].cpu().numpy()  # Access the first element and move to CPU for numpy

    output = tabnet_model.network(X_test_tensor.to(tabnet_model.device))
    test_features = output[0].cpu().numpy() # Access the first element and move to CPU for numpy


# Train SVM on the extracted features
svm_model = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
svm_model.fit(train_features, y_train)

# Make predictions
y_pred = svm_model.predict(test_features)
y_pred_proba = svm_model.predict_proba(test_features)[:, 1]

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC Score: {roc_auc:.3f}")

# Feature importance from TabNet
feat_importances = tabnet_model.feature_importances_
feat_importance_dict = dict(zip(top_features, feat_importances))
sorted_feat_importances = sorted(feat_importance_dict.items(), key=lambda x: x[1], reverse=True)

print("\nTabNet Feature Importances:")
for feat, importance in sorted_feat_importances:
    print(f"{feat}: {importance:.6f}")


Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.70      0.82     19146
           1       0.10      0.78      0.18       854

    accuracy                           0.70     20000
   macro avg       0.54      0.74      0.50     20000
weighted avg       0.95      0.70      0.79     20000


Confusion Matrix:
[[13362  5784]
 [  187   667]]

ROC AUC Score: 0.808

TabNet Feature Importances:
Duration: 0.285232
Total.miles.driven: 0.184323
Avgdays.week: 0.164837
Annual.pct.driven: 0.113711
Brake.06miles: 0.065162
Pct.drive.sat: 0.038803
Years.noclaims: 0.033412
Credit.score: 0.031820
Pct.drive.2hrs: 0.023103
Pct.drive.sun: 0.021995
Pct.drive.tue: 0.014535
Pct.drive.rush am: 0.013778
Car.age: 0.006937
Pct.drive.mon: 0.002280
Accel.06miles: 0.000035
Pct.drive.rush pm: 0.000033
Pct.drive.thr: 0.000003
Pct.drive.fri: 0.000001
Pct.drive.wed: 0.000001
Left.turn.intensity09: 0.000000
